In [1]:
%load_ext autoreload

%autoreload 2

In [3]:
import main_agent as ma
import prompts_lib as pl


In [5]:
message = pl.system_prompt2 + 'How many studio albums were published by Mercedes Sousa between 2000 and 2009 included?'

In [18]:
from langfuse.langchain import CallbackHandler

# Initialize Langfuse CallbackHandler for LangGraph/Langchain (tracing)
langfuse_handler = CallbackHandler()
langfuse_handler2 = CallbackHandler()

class BasicAgent:
    def __init__(self):
        self.agent = ma.agent
    def __call__(self, question: str) -> str:
        print(f"Agent received question (first 50 chars): {question[:50]}...")

        message = pl.system_prompt2 + question

        response = self.agent.invoke(
            input={'messages': message},
            config={"callbacks": [langfuse_handler2]}
            )

        answer = response.get('messages')[1].content

        # fixed_answer = "This is a default answer."
        print(f"Agent returning answer: {answer}")
        return answer

In [19]:
agent = BasicAgent()

In [20]:
answer = agent("How many studio albums were published by Mercedes Sousa between 2000 and 2009 included? use duckduckgo tool you have available to search wikipedia to respond")

Agent received question (first 50 chars): How many studio albums were published by Mercedes ...
Agent returning answer: Seven
